# Manejo de Datasets con Pandas

Este notebook tiene como objetivo introducir el manejo de **datasets con la librería Pandas**,
simulando operaciones similares a las de una base de datos SQL: `SELECT`, `WHERE`, `JOIN`, `GROUP BY`.

**Objetivo:** aprender a cargar, filtrar, unir y agrupar datos usando Pandas,
aplicando estos conocimientos a modelos de datos como asistencias, stock o consultas médicas.


## Importación de Pandas y carga de datos

Primero importamos la librería Pandas y creamos algunos datasets de ejemplo.
<BR>
Vamos a seguir en línea con el schema de Postgres, una tabla para estudiantes y otra para asistencias.


En caso de tener que importar un archivo 'csv' o 'json' pueden hacerlo con:

* df = pd.read_csv("ejemplo.csv")
* df = pd.read_json("archivo.json")

In [1]:
# Importar Pandas
import pandas as pd

In [2]:
# Creamos el dataset estudiante con algunos datos:
estudiantes = pd.DataFrame({
    'id_estudiante': [1, 2, 3,],
    'nombre': ['Ana', 'Luis', "Sofía"],
    'apellido': ['Kunst', 'Martins', 'Ripol'],
    'legajo': ['1568', '2649', '1523']
})

estudiantes

,id_estudiante,nombre,apellido,legajo
0,1,Ana,Kunst,1568
1,2,Luis,Martins,2649
2,3,Sofía,Ripol,1523


In [3]:
# Creamos el dataset asistencias con algunos datos:
asistencias = pd.DataFrame({
    'id_asistencia': [101, 102, 103, 104, 105],
    'id_estudiante': [1, 2, 1, 3, 2],
    'asignatura': ['Algoritmos', 'Marketing', 'Algoritmos', 'Progra I', 'Marketing'],
    'fecha': ['2025-03-01', '2025-04-01', '2025-04-09', '2025-05-08', '2025-05-15'],
    'estado': ["P", "A", "M", "P", "P"]
})

asistencias

,id_asistencia,id_estudiante,asignatura,fecha,estado
0,101,1,Algoritmos,2025-03-01,P
1,102,2,Marketing,2025-04-01,A
2,103,1,Algoritmos,2025-04-09,M
3,104,3,Progra I,2025-05-08,P
4,105,2,Marketing,2025-05-15,P


## 🔍 Exploración básica de DataFrames

Podemos inspeccionar la estructura y contenido de los DataFrames.


In [14]:
# Primeras filas
estudiantes.head(2)

,id_estudiante,nombre,apellido,legajo
0,1,Ana,Kunst,1568
1,2,Luis,Martins,2649


In [16]:
# Información general
asistencias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id_asistencia  5 non-null      int64 
 1   id_estudiante  5 non-null      int64 
 2   asignatura     5 non-null      object
 3   fecha          5 non-null      object
 4   estado         5 non-null      object
dtypes: int64(2), object(3)
memory usage: 332.0+ bytes


In [17]:
# Listar las columnas
estudiantes.columns

Index(['id_estudiante', 'nombre', 'apellido', 'legajo'], dtype='object')

## Análisis y conversión de tipo de datos

### dtypes

In [18]:
# A diferencia de Python nativo, en Pandas podemos forzar un tipo de dato.
# La propiedad es dtype
asistencias.dtypes

id_asistencia     int64
id_estudiante     int64
asignatura       object
fecha            object
estado           object
dtype: object

### astype()

In [20]:
# Convertir str a int o int a str con astype
asistencias['id_estudiante'] = asistencias['id_estudiante'].astype(int)
asistencias.dtypes
#astype() admite, int, float, str

id_asistencia     int64
id_estudiante     int64
asignatura       object
fecha            object
estado           object
dtype: object

### to_datetime()

In [ ]:
# Vamos a convertir el dtype de fecha.
#asistencias['fecha'].dtype

#asistencias['fecha'].sample() # YYY-MM-DD

asistencias['fecha'] = pd.to_datetime(asistencias['fecha'])
# se admite format="%d/%m/%Y", erros="coerce"

asistencias['fecha'].dtype

dtype('<M8[ns]')

## Selección y filtrado de columnas y filas

Simulamos un `SELECT` y un `WHERE` en SQL.


In [26]:
# Seleccionar columnas específicas
estudiantes[['id_estudiante', 'apellido']]


,id_estudiante,apellido
0,1,Kunst
1,2,Martins
2,3,Ripol


In [27]:
# Identifcar valores únicos
asistencias['asignatura'].unique()

array(['Algoritmos', 'Marketing', 'Progra I'], dtype=object)

In [28]:
# Contar valores unicos
asistencias['asignatura'].value_counts()

asignatura
Algoritmos    2
Marketing     2
Progra I      1
Name: count, dtype: int64

In [29]:
# Filtrar filas (WHERE asignatura = 'Matemática Discreta')
asistencias[asistencias['asignatura'] == 'Marketing']


,id_asistencia,id_estudiante,asignatura,fecha,estado
1,102,2,Marketing,2025-04-01,A
4,105,2,Marketing,2025-05-15,P


In [30]:
# Filtrar filas (WHERE asignatura similar 'Matemática Discreta')
asistencias[asistencias['asignatura'].str.lower().str.contains("mark", case=False)]

,id_asistencia,id_estudiante,asignatura,fecha,estado
1,102,2,Marketing,2025-04-01,A
4,105,2,Marketing,2025-05-15,P


In [32]:
# Filtrar con query
asistencias.query("asignatura == 'Marketing'")



,id_asistencia,id_estudiante,asignatura,fecha,estado
1,102,2,Marketing,2025-04-01,A
4,105,2,Marketing,2025-05-15,P


In [34]:
asistencias.loc[asistencias["estado"] == "A", ["asignatura"]]

,asignatura
1,Marketing


In [36]:
asistencias[["asignatura", "fecha"]]

,asignatura,fecha
0,Algoritmos,2025-03-01
1,Marketing,2025-04-01
2,Algoritmos,2025-04-09
3,Progra I,2025-05-08
4,Marketing,2025-05-15


In [40]:
asistencias['fecha'].dt.year

0    2025
1    2025
2    2025
3    2025
4    2025
Name: fecha, dtype: int32

In [43]:
# Acceder al mes o año para filtros mas amplios
asistencias[asistencias['fecha'].dt.month >= 4]

,id_asistencia,id_estudiante,asignatura,fecha,estado
1,102,2,Marketing,2025-04-01,A
2,103,1,Algoritmos,2025-04-09,M
3,104,3,Progra I,2025-05-08,P
4,105,2,Marketing,2025-05-15,P


In [ ]:
# Acceder al mes o año para filtros mas amplios con &
asistencias[(asistencias['fecha'].dt.month == 4) & (asistencias['asignatura']=='Algoritmos')]

## 📊 Ordenamiento de datos

Simula un `ORDER BY`.


In [ ]:
# Ordenar estudiantes por nombre
estudiantes.sort_values('nombre')


In [46]:
# Ordenar asistencias por asignatura descendiente
asistencias.sort_values(by="asignatura", ascending=False)

,id_asistencia,id_estudiante,asignatura,fecha,estado
3,104,3,Progra I,2025-05-08,P
1,102,2,Marketing,2025-04-01,A
4,105,2,Marketing,2025-05-15,P
0,101,1,Algoritmos,2025-03-01,P
2,103,1,Algoritmos,2025-04-09,M


In [48]:
# Ordenar por asignatura y por estado
asistencias.sort_values(['asignatura', 'fecha'])


,id_asistencia,id_estudiante,asignatura,fecha,estado
0,101,1,Algoritmos,2025-03-01,P
2,103,1,Algoritmos,2025-04-09,M
1,102,2,Marketing,2025-04-01,A
4,105,2,Marketing,2025-05-15,P
3,104,3,Progra I,2025-05-08,P


## 🔗 Combinación de DataFrames (JOIN)

Simula un `JOIN` entre tablas SQL.


In [49]:
# INNER JOIN: estudiantes con sus asistencias
join_ea = pd.merge(estudiantes, asistencias, on='id_estudiante', how='inner')
join_ea


,id_estudiante,nombre,apellido,legajo,id_asistencia,asignatura,fecha,estado
0,1,Ana,Kunst,1568,101,Algoritmos,2025-03-01,P
1,1,Ana,Kunst,1568,103,Algoritmos,2025-04-09,M
2,2,Luis,Martins,2649,102,Marketing,2025-04-01,A
3,2,Luis,Martins,2649,105,Marketing,2025-05-15,P
4,3,Sofía,Ripol,1523,104,Progra I,2025-05-08,P


## 📈 Agrupamiento y agregación (GROUP BY)

Simula un `GROUP BY` de SQL.


In [53]:
# Cantidad de asistencias por carrera
# Primero filtramos los presentes
reporte = join_ea[join_ea['estado'] == 'P']
#reporte = reporte.groupby('asignatura')['estado'].count().reset_index()
reporte

,id_estudiante,nombre,apellido,legajo,id_asistencia,asignatura,fecha,estado
0,1,Ana,Kunst,1568,101,Algoritmos,2025-03-01,P
3,2,Luis,Martins,2649,105,Marketing,2025-05-15,P
4,3,Sofía,Ripol,1523,104,Progra I,2025-05-08,P


In [52]:
reporte.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, 0 to 4
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id_estudiante  3 non-null      int64         
 1   nombre         3 non-null      object        
 2   apellido       3 non-null      object        
 3   legajo         3 non-null      object        
 4   id_asistencia  3 non-null      int64         
 5   asignatura     3 non-null      object        
 6   fecha          3 non-null      datetime64[ns]
 7   estado         3 non-null      object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 216.0+ bytes


In [54]:
# Cantidad de asistencias por estudiante
# Primero filtramos los presentes
reporte = join_ea[join_ea['estado'] == 'P']
reporte = reporte.groupby(['nombre', 'apellido'], as_index=False)['estado'].count().reset_index()
reporte

,index,nombre,apellido,estado
0,0,Ana,Kunst,1
1,1,Luis,Martins,1
2,2,Sofía,Ripol,1


In [55]:
# Cantidad de asistencias por estudiante (usando otros métodos de agregación)
# Primero filtramos los presentes
reporte = join_ea[asistencias['estado'] == 'P']
reporte = reporte.groupby(['nombre', 'apellido'], as_index=False).agg(
    asistencia = ('estado', 'count')
)
# reporte = reporte.groupby(['nombre', 'apellido'], as_index=False).agg(
#     {'estado': 'count'}
# )
reporte

,nombre,apellido,asistencia
0,Ana,Kunst,1
1,Luis,Martins,1
2,Sofía,Ripol,1


## 🧩 Ejercicio final

1. Crea DataFrames para otro modelo (por ejemplo, *productos*, *proveedores* y *stock*).
2. Usa `merge()` para generar un reporte que relacione productos y proveedores.
3. Usa `groupby()` para obtener el total de stock por proveedor.
4. Guarda el resultado en un archivo CSV con `to_csv('reporte.csv', index=False)`.
